In [ ]:
import time
import requests
import json
from datetime import datetime
from pathlib import Path

API_BASE = "https://fair2-dev.lutechdigitale.it/med-gemma"
PREDICTION_API_BASE = "https://fair2-dev.lutechdigitale.it/data-provisioning-brainmed"

# Configurazioni predefinite ottimizzate
CONFIGS = {
    'conservative': {
        'temperature': 0.1,
        'top_p': 0.3,
        'max_tokens': 500,
        'do_sample': True,
        'n_runs': 1,
        'validation_threshold': 40
    },
    'balanced': {
        'temperature': 0.2,
        'top_p': 0.5,
        'max_tokens': 700,
        'do_sample': True,
        'n_runs': 1,
        'validation_threshold': 50
    },
    'creative': {
        'temperature': 0.3,
        'top_p': 0.7,
        'max_tokens': 700,
        'do_sample': True,
        'n_runs': 1,
        'validation_threshold': 45
    }
}

# Usa configurazione conservativa per default (più stabile)
CURRENT_CONFIG = CONFIGS['conservative']

# ==================== PREDICTION API SERVICE ====================

class BrainMedPredictionService:
    def __init__(self, base_url: str = PREDICTION_API_BASE):
        self.base_url = base_url
    
    def get_patient_prediction(self, patient_id: str) -> dict | None:
        """
        Recupera le predizioni e metriche per un paziente specifico
        
        Args:
            patient_id: ID del paziente
            
        Returns:
            Dict con predizioni e metriche o None se errore
        """
        url = f"{self.base_url}/prediction_model/{patient_id}/latest"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ Errore nella chiamata API predizioni: {e}")
            return None
    
    def format_prediction_for_prompt(self, prediction_data: dict, include_metrics: bool = True) -> str:
        """
        Formatta i dati di predizione per il prompt AI
        
        Args:
            prediction_data: Dati dalla API
            include_metrics: Se includere le metriche nel prompt
            
        Returns:
            Stringa formattata per il prompt
        """
        if not prediction_data:
            return "❌ Nessuna predizione disponibile per questo paziente."
        
        # Mappiamo la classe della malattia a una descrizione
        disease_classes = {
            0: "Nessuna patologia rilevata (CN - Cognitivamente Normale)",
            1: "Patologia rilevata (possibile MCI o AD)",
            # Aggiungi altre classi se necessario
        }
        
        class_disease = prediction_data.get('class_disease', 'N/A')
        disease_description = disease_classes.get(class_disease, f"Classe {class_disease}")
        
        prompt_text = f"""
🤖 PREDIZIONE MODELLO AI:
• Classe predetta: {disease_description}
• Valore numerico: {class_disease}
• Data predizione: {prediction_data.get('date_time', 'N/A')}
• ID paziente: {prediction_data.get('id', 'N/A')}
"""
        
        if include_metrics:
            accuracy = prediction_data.get('accuracy', 0)
            precision = prediction_data.get('precision', 0)
            recall = prediction_data.get('recall', 0)
            f1_score = prediction_data.get('f1_score', 0)
            
            prompt_text += f"""
📊 METRICHE DEL MODELLO:
• Accuracy: {accuracy:.1%} (precisione generale)
• Precision: {precision:.1%} (quando predice positivo, quanto è accurato)
• Recall: {recall:.1%} (quanti casi positivi cattura)
• F1 Score: {f1_score:.1%} (bilanciamento precision/recall)
"""
        
        return prompt_text.strip()

# ==================== EXISTING FUNCTIONS ====================

def predict_image(
    image_path: str,
    prompt: str,
    *,
    max_tokens: int = 500,
    temperature: float = 0.1,
    top_p: float = 0.3,
    num_beams: int | None = None,
    do_sample: bool = True,
    poll_interval: int = 5,
    timeout: int = 900,
):
    """Funzione per chiamare l'API di predizione"""
    with open(image_path, "rb") as f:
        files = {"file": (image_path, f, "application/octet-stream")}
        data = {
            "prompt": prompt,
            "max_tokens": str(max_tokens),
            "temperature": str(temperature),
            "top_p": str(top_p),
            **({"num_beams": str(num_beams)} if num_beams is not None else {}),
            "do_sample": str(do_sample).lower(),
        }

        resp = requests.post(
            f"{API_BASE}/predict_image",
            files=files,
            data=data,
            timeout=(10, 30),
        )
    resp.raise_for_status()
    job_id = resp.json()["job_id"]

    start = time.time()
    while True:
        if time.time() - start > timeout:
            raise TimeoutError("Timeout waiting for job to finish")

        status_resp = requests.get(
            f"{API_BASE}/jobs/{job_id}", timeout=(5, 10)
        )
        status_resp.raise_for_status()
        payload = status_resp.json()
        status = payload["status"]

        if status == "pending":
            time.sleep(poll_interval)
            continue
        elif status == "done":
            return payload["result"]
        else:
            raise RuntimeError(f"Job failed: {payload.get('detail')}")

def clean_response_v2(text: str) -> str:
    """Pulisce la risposta da ripetizioni e artefatti"""
    # Rimuovi ripetizioni del prompt
    if "Do not repeat these instructions" in text:
        text = text.split("Do not repeat these instructions")[0]
    
    # Rimuovi ripetizioni evidenti (stesso paragrafo ripetuto)
    lines = text.split('\n')
    seen_lines = set()
    cleaned_lines = []
    
    for line in lines:
        line = line.strip()
        if line and line not in seen_lines:
            seen_lines.add(line)
            cleaned_lines.append(line)
        elif not line:  # Mantieni righe vuote
            cleaned_lines.append(line)
    
    text = '\n'.join(cleaned_lines)
    
    # Trova inizio analisi se presente
    start_markers = ["Primary Findings:", "What I see:", "Describe what you see", "Findings:", "OSSERVAZIONI CONCRETE:", "COSA VEDO:"]
    start_index = -1
    
    for marker in start_markers:
        idx = text.find(marker)
        if idx != -1:
            start_index = idx
            break
    
    if start_index != -1:
        # Cerca ripetizioni del marker
        second_index = text.find(marker, start_index + 20)
        if second_index != -1:
            return text[start_index:second_index].strip()
        return text[start_index:].strip()
    
    return text.strip()

def validate_actual_analysis(result: str) -> bool:
    """Valida che il risultato contenga analisi reale, non template"""
    # Verifica che non sia solo template
    template_indicators = [
        '[Specific finding',
        '[Describe',
        '[Identify',
        '[High/Medium/Low]',
        '[Immediate clinical',
        '[Clinical significance',
        '[Activation intensity'
    ]
    
    # Se contiene troppi indicatori di template, è fallito
    template_count = sum(1 for indicator in template_indicators if indicator in result)
    if template_count > 1:
        return False
    
    # Verifica che non sia ripetitivo
    lines = result.split('\n')
    unique_lines = set(line.strip() for line in lines if line.strip())
    if len(lines) > 10 and len(unique_lines) < len(lines) * 0.7:
        return False
    
    # Verifica presenza di contenuto medico specifico
    medical_content = [
        'activation', 'cortex', 'hippocampus', 'temporal', 'frontal',
        'red', 'blue', 'high', 'low', 'medium', 'pattern', 'region',
        'sagittale', 'coronale', 'assiale', 'slice'
    ]
    
    content_count = sum(1 for term in medical_content if term.lower() in result.lower())
    return content_count >= 3

def calculate_real_quality_score(result: str) -> int:
    """Score basato su contenuto reale, non formale"""
    score = 0
    
    # Penalizza heavily se è solo template
    if not validate_actual_analysis(result):
        return 0
    
    # Penalizza ripetizioni
    lines = result.split('\n')
    unique_lines = set(line.strip() for line in lines if line.strip())
    if len(lines) > 5:
        repetition_ratio = len(unique_lines) / len([l for l in lines if l.strip()])
        score += int(repetition_ratio * 50)
    
    # Premia contenuto medico specifico
    medical_terms = [
        'cortex', 'hippocampus', 'temporal', 'parietal', 'frontal',
        'activation', 'heatmap', 'grad-cam', 'alzheimer', 'pathology'
    ]
    
    for term in medical_terms:
        if term.lower() in result.lower():
            score += 15
    
    # Premia descrizioni di colori/pattern
    visual_terms = ['red', 'blue', 'yellow', 'green', 'bright', 'dark', 'pattern', 'region']
    for term in visual_terms:
        if term.lower() in result.lower():
            score += 10
    
    # Premia menzioni di slice/axis
    slice_terms = ['slice', 'sagittal', 'coronal', 'axial', 'x-axis', 'y-axis', 'z-axis', 'sagittale', 'coronale', 'assiale']
    for term in slice_terms:
        if term.lower() in result.lower():
            score += 20
    
    # Premia struttura senza template
    if any(section in result for section in ['Primary Findings:', 'What I see:', 'Alzheimer', 'OSSERVAZIONI', 'COSA VEDO']) and '[' not in result:
        score += 30
    
    # Premia analisi multi-slice
    multi_slice_indicators = ['sagittale', 'coronale', 'assiale', 'x=64', 'y=64', 'z=25']
    multi_slice_count = sum(1 for indicator in multi_slice_indicators if indicator.lower() in result.lower())
    if multi_slice_count >= 2:
        score += 25
    
    return score

def format_long_text(text: str, max_line_length: int = 80) -> str:
    """Formatta testo lungo andando a capo"""
    existing_lines = text.split('\n')
    formatted_lines = []
    
    for line in existing_lines:
        line = line.strip()
        if not line:
            formatted_lines.append("")
            continue
            
        if len(line) <= max_line_length:
            formatted_lines.append(line)
            continue
            
        words = line.split()
        current_line = ""
        
        for word in words:
            if len(current_line + " " + word) <= max_line_length:
                current_line += " " + word if current_line else word
            else:
                if current_line:
                    formatted_lines.append(current_line)
                current_line = word
        
        if current_line:
            formatted_lines.append(current_line)
    
    return "\n".join(formatted_lines)

def get_unique_filename(output_dir: Path, image_path: str) -> Path:
    """Genera nome file unico basato sull'immagine"""
    image_name = Path(image_path).stem
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    return output_dir / f"{image_name}_{timestamp}.jsonl"

def save_result_to_jsonl(image_path, prompt, cleaned_result, quality_score=None, prediction_data=None):
    """Salva risultato in formato JSONL con predizioni integrate"""
    try:
        output_dir = Path(r"C:\Users\nicolo.petruzzella\OneDrive - LUTECH SPA\Desktop\promptMRI\outputs_jsonl")
        output_dir.mkdir(parents=True, exist_ok=True)
        
        output_file = get_unique_filename(output_dir, image_path)

        formatted_result = format_long_text(cleaned_result, max_line_length=80)
        formatted_prompt = format_long_text(prompt, max_line_length=80)
        
        record = {
            "timestamp": datetime.now().isoformat(),
            "image": image_path,
            "prompt": formatted_prompt,
            "result": formatted_result,
            "quality_score": quality_score,
            "config": CURRENT_CONFIG,
            "prediction_data": prediction_data  # Aggiungi predizioni
        }

        with open(output_file, "w", encoding="utf-8") as f:
            f.write("{\n")
            f.write(f'  "timestamp": "{record["timestamp"]}",\n')
            f.write(f'  "image": "{record["image"]}",\n')
            f.write('  "prompt": "')
            f.write(record["prompt"].replace('"', '\\"').replace('\n', '\\n'))
            f.write('",\n')
            f.write('  "result": "')
            f.write(record["result"].replace('"', '\\"').replace('\n', '\\n'))
            f.write('",\n')
            f.write(f'  "quality_score": {quality_score},\n')
            f.write('  "config": ')
            f.write(json.dumps(record["config"]))
            f.write(',\n')
            f.write('  "prediction_data": ')
            f.write(json.dumps(record["prediction_data"]))
            f.write('\n}\n')
        
        print(f"✅ Risultato salvato in: {output_file.name}")
            
    except Exception as e:
        print(f"❌ Errore durante il salvataggio: {e}")

# ==================== PROMPT FUNCTIONS WITH PREDICTIONS ====================

def get_multi_slice_optimized_prompt_with_predictions(prediction_info: str = ""):
    """Prompt ottimizzato per analisi multi-slice con predizioni AI integrate"""
    base_prompt = (
        "Stai analizzando uno screenshot che contiene 3 slice MRI cerebrali con sovrapposizione Grad-CAM per rilevazione Alzheimer.\n\n"
        "IMPORTANTE: Analizza l'IMMAGINE REALE che vedi, non ripetere le istruzioni.\n\n"
    )
    
    if prediction_info:
        base_prompt += f"{prediction_info}\n\n"
    
    base_prompt += (
        "Contesto tecnico:\n"
        "- L'immagine mostra 3 sezioni ortogonali del cervello:\n"
        "    • Sagittale (x-axis, coordinata X=64): visione laterale del cervello, utile per analizzare strutture profonde come ippocampo e corpo calloso\n"
        "    • Coronale (y-axis, coordinata Y=64): visione frontale, cruciale per confrontare emisferi destro/sinistro e osservare simmetrie\n"
        "    • Assiale (z-axis, coordinata Z=25): visione dall'alto/inferiore, utile per identificare distribuzioni corticali\n"
        "- Rosso/caldo = alta attivazione del modello\n"
        "- Blu/freddo = bassa attivazione\n\n"
        "Fornisci la tua analisi seguendo ESATTAMENTE questa struttura:\n\n"
        "🧠 Primary Findings:\n"
        "• Descrivi quello che vedi effettivamente in ciascuna delle 3 slice\n"
        "• Specifica colori, intensità e posizione anatomica precisa\n"
        "• Nota differenze significative tra le slice\n\n"
        "🔬 Alzheimer's Indicators:\n"
        "• Le attivazioni interessano ippocampo, corteccia entorinale, o aree tipiche AD?\n"
        "• Il pattern è coerente con neurodegenerazione?\n"
        "• Ci sono asimmetrie tra emisferi? In quale piano?\n"
    )
    
    if prediction_info:
        base_prompt += "• Come si allinea l'analisi visiva con la predizione del modello AI?\n"
    
    base_prompt += (
        "\n📈 Confidence Level:\n"
        "• Alto/Medio/Basso - Spiega basandoti su chiarezza dell'immagine e pattern\n"
    )
    
    if prediction_info:
        base_prompt += "• Considera anche l'affidabilità delle metriche del modello AI\n"
    
    base_prompt += (
        "\n📌 Recommended Actions:\n"
        "• Altre slice necessarie? Specifica asse e coordinate\n"
        "• Esami clinici aggiuntivi suggeriti\n"
        "• Prossimi step diagnostici\n\n"
        "Descrivi solo quello che vedi realmente nell'immagine."
    )
    
    return base_prompt

def get_multi_slice_robust_prompt_v2_with_predictions(prediction_info: str = ""):
    """Prompt robusto con predizioni integrate"""
    base_prompt = (
        "Stai osservando un'immagine che mostra 1, 2 o 3 slice MRI cerebrali con sovrapposizione Grad-CAM, usate per l'analisi di Alzheimer.\n\n"
    )
    
    if prediction_info:
        base_prompt += f"{prediction_info}\n\n"
    
    base_prompt += (
        "🧠 Attenzione:\n"
        "- L'immagine attuale mostra slice alle seguenti coordinate:\n"
        "   • Sagittale (x-axis): x = 64\n"
        "   • Coronale (y-axis): y = 64\n"
        "   • Assiale (z-axis): z = 25\n"
        "- Le coordinate possibili in questo esame sono:\n"
        "   • x ∈ [0, 127] (sagittale)\n"
        "   • y ∈ [0, 127] (coronale)\n"
        "   • z ∈ [0, 49]  (assiale)\n"
        "- Ogni slice rappresenta una vista ortogonale del cervello:\n"
        "   • Sagittale (x): visione laterale\n"
        "   • Coronale (y): visione frontale\n"
        "   • Assiale (z): visione dall'alto o inferiore\n"
        "- Colori caldi (rosso/giallo) = alta attivazione del modello\n"
        "- Colori freddi (blu) = bassa attivazione\n\n"
        "✏️ Analizza seguendo questa struttura, ma **usa parole tue**:\n\n"
        "🔍 Slice Analysis:\n"
        "• Quante slice vedi chiaramente?\n"
        "• Che orientamento sembrano avere (x, y, z)? Dove sono posizionate visivamente?\n"
        "• Cosa mostrano a livello di attivazioni?\n\n"
        "🧠 Alzheimer's Indicators:\n"
        "• Le attivazioni coinvolgono regioni come ippocampo o corteccia entorinale?\n"
        "• Il pattern è coerente con Alzheimer o altre forme di neurodegenerazione?\n"
        "• Noti asimmetrie tra emisferi?\n"
    )
    
    if prediction_info:
        base_prompt += "• L'analisi visiva conferma o contraddice la predizione del modello?\n"
    
    base_prompt += (
        "\n📈 Confidence Level:\n"
        "• Valuta il tuo livello di sicurezza (Alto / Medio / Basso) e spiega il perché\n"
    )
    
    if prediction_info:
        base_prompt += "• Considera l'affidabilità delle metriche AI nella tua valutazione\n"
    
    base_prompt += (
        "\n📌 Azioni Consigliate:\n"
        "• Servono altre slice? Di quale asse e a quale coordinata (se lo ritieni utile)?\n"
        "• Se opportuno, suggerisci anche coordinate alternative rispetto a quelle mostrate (es. x, y, z diverse), motivando la scelta.\n"
        "• Suggerisci esami clinici o neuroimaging aggiuntivi\n"
        "• Indica i prossimi passi per una diagnosi più precisa\n\n"
        "🧠 **Stima dello stadio della malattia:**\n"
        "• Sulla base delle attivazioni visibili"
    )
    
    if prediction_info:
        base_prompt += " e della predizione del modello"
    
    base_prompt += (
        ", indica in quale fase potremmo trovarci:\n"
        "   - CN (Cognitivamente Normale)\n"
        "   - MCI (Mild Cognitive Impairment)\n"
        "   - s-MCI (MCI stabile)\n"
        "   - c-MCI (MCI con progressione verso Alzheimer)\n"
        "   - AD (Malattia di Alzheimer conclamata)\n"
        "• Motiva brevemente la tua valutazione.\n\n"
        "❗Non ripetere queste istruzioni. Concentrati solo sull'immagine reale che stai vedendo."
    )
    
    return base_prompt

def select_prompt_with_predictions(prediction_info: str = ""):
    """Seleziona quale prompt usare, con predizioni integrate"""
    prompts = {
        "1": ("Multi-Slice Ottimizzato CON Predizioni (RACCOMANDATO)", 
              get_multi_slice_optimized_prompt_with_predictions(prediction_info)),
        "2": ("Diretto Visuale CON Predizioni (v2)", 
              get_multi_slice_robust_prompt_v2_with_predictions(prediction_info)),
        "3": ("Multi-Slice Ottimizzato SENZA Predizioni", 
              get_multi_slice_optimized_prompt_with_predictions("")),
        "4": ("Diretto Visuale SENZA Predizioni", 
              get_multi_slice_robust_prompt_v2_with_predictions("")),
    }
    
    print("🧠 Seleziona il prompt da usare:")
    for key, (name, _) in prompts.items():
        print(f"{key}. {name}")
    
    choice = input("\nScegli (1-4, default=1): ").strip()
    
    if choice not in prompts:
        choice = "1"  # Default al primo
    
    selected_name, selected_prompt = prompts[choice]
    print(f"✅ Selezionato: {selected_name}\n")
    
    return selected_prompt

# ==================== MAIN EXECUTION ====================

def main_with_predictions():
    """Main function integrata con API predizioni"""
    # Immagine di test
    img_file = "C:/Users/nicolo.petruzzella/OneDrive - LUTECH SPA/Desktop/promptMRI/imagesPrompt/z/Screenshot 2025-07-10 112906.png"
    
    print("🧠 Analisi MRI con Grad-CAM per Alzheimer (CON PREDIZIONI AI)\n")
    
    # Inizializza servizio predizioni
    prediction_service = BrainMedPredictionService()
    
    # ID paziente di esempio (sostituisci con quello reale)
    patient_id = "40a7bc47-e0ca-41ec-bb17-467957548be0"
    
    # Opzione per includere predizioni
    use_predictions = input("🤖 Vuoi includere le predizioni AI? (y/n, default=y): ").strip().lower()
    if use_predictions == '' or use_predictions == 'y':
        print(f"📡 Recupero predizioni per paziente: {patient_id}")
        prediction_data = prediction_service.get_patient_prediction(patient_id)
        
        if prediction_data:
            print("✅ Predizioni recuperate!")
            print(f"🎯 Classe predetta: {prediction_data.get('class_disease', 'N/A')}")
            print(f"📊 Accuracy: {prediction_data.get('accuracy', 0):.1%}")
            
            # Opzione per includere metriche
            include_metrics = input("📊 Includere metriche nel prompt? (y/n, default=y): ").strip().lower()
            include_metrics = include_metrics == '' or include_metrics == 'y'
            
            prediction_info = prediction_service.format_prediction_for_prompt(
                prediction_data, include_metrics
            )
        else:
            print("❌ Impossibile recuperare predizioni. Procedo senza.")
            prediction_info = ""
            prediction_data = None
    else:
        prediction_info = ""
        prediction_data = None
    
    # Selezione prompt con predizioni
    selected_prompt = select_prompt_with_predictions(prediction_info)
    
    # Selezione configurazione
    print("⚙️ Seleziona configurazione:")
    for key, config in CONFIGS.items():
        print(f"- {key}: temp={config['temperature']}, tokens={config['max_tokens']}")
    
    config_choice = input("\nScegli configurazione (conservative/balanced/creative, default=conservative): ").strip()
    if config_choice in CONFIGS:
        global CURRENT_CONFIG
        CURRENT_CONFIG = CONFIGS[config_choice]
        print(f"✅ Configurazione: {config_choice}")
    else:
        print("✅ Configurazione: conservative (default)")
    
    print(f"\n🔧 Parametri: temp={CURRENT_CONFIG['temperature']}, tokens={CURRENT_CONFIG['max_tokens']}")
    print("-" * 60)
    
    try:
        result = predict_image(
            img_file,
            selected_prompt,
            max_tokens=CURRENT_CONFIG['max_tokens'],
            temperature=CURRENT_CONFIG['temperature'],
            top_p=CURRENT_CONFIG['top_p'],
            do_sample=CURRENT_CONFIG['do_sample']
        )
        
        result_clean = clean_response_v2(result)
        
        # Validazione
        is_valid = validate_actual_analysis(result_clean)
        quality_score = calculate_real_quality_score(result_clean)
        
        print(f"📊 Risultato:")
        print(f"✅ Valido: {is_valid}")
        print(f"🏆 Score: {quality_score}")
        print(f"📏 Lunghezza: {len(result_clean)} caratteri")
        
        if prediction_data:
            print(f"🤖 Predizione AI: Classe {prediction_data.get('class_disease', 'N/A')}")
            print(f"📊 Accuracy: {prediction_data.get('accuracy', 0):.1%}")
        
        print(f"\n📋 Analisi completa:")
        print("-" * 60)
        print(result_clean)
        print("-" * 60)
        
        # Salvataggio con predizioni
        save_result_to_jsonl(img_file, selected_prompt, result_clean, quality_score, prediction_data)
        
    except Exception as e:
        print(f"❌ Errore durante l'analisi: {e}")

if __name__ == "__main__":
    main_with_predictions()